## Videos to video, bbox with class color, label, score, IoU

In [ ]:
from tqdm import tqdm
from PIL import Image
import numpy as np
import cv2
import json
import os

from utils import torch_utils, painter

###### config
OUTPUT_DIR = '../output/video-output'
N_CLASSES = 6

VIDEO_LIST = [
    #'/path/to/your/video.mp4',
    '../video/n7141/[CH04] 2019-10-08 20.55.00 10min.mp4'
]
TRACK_LIST = [
    #'/path/to/your/raw_track.json',
    '../output/raw-tracks/n7141/[CH04] 2019-10-08 20.55.00 10min.json'
]

######


if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

color_chart = painter.set_colors(N_CLASSES, saturation=0.8) 

for v_path, t_path in zip(VIDEO_LIST, TRACK_LIST):
    with open(t_path) as jf:
        data = json.load(jf)
        vid_info = data['Video']
        tracks = data['Tracks']
        
    vid = cv2.VideoCapture(v_path)
    vid_name = os.path.basename(v_path)
    
    filename = f'{OUTPUT_DIR}/output-{os.path.splitext(vid_name)[0]}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = vid_info['FPS']
    wh = (vid_info['Width'], vid_info['Height'])
    
    cam = cv2.VideoWriter(filename, fourcc, fps, wh)
    
    with tqdm(desc=vid_name, total=vid_info['Length']) as pbar:
        while vid.isOpened():

            f_no = int(vid.get(1))
            success, frame = vid.read()
            if not success:
                break

            if vid_info['Crop 4x3']:
                frame = torch_utils.center_crop_4x3_numpy(frame)

            colors = [color_chart[i] for i in tracks[f_no]['Label IDs']]
            
            tags = [
                f'{l} c{c:.2f} i{i:.2f}' for l, c, i in zip(tracks[f_no]['Labels'], tracks[f_no]['Label Scores'], tracks[f_no]['IoU Scores'])
            ]
            
            
            frame = Image.fromarray(frame)
            frame = painter.mark_boxes(
                frame, 
                tracks[f_no]['Boxes'], 
                tags = tags,
                colors = colors,
                font_path='data/simhei.ttf',
                filling = tracks[f_no]['On Mouse'],
                filling_color = colors
            ).convert('RGB')

            frame = np.array(frame)
            cam.write(frame)
            pbar.update(1)
            
    cam.release()

## Frames to video, bbox without class color

In [1]:
from glob import glob
import random
import cv2
from PIL import Image
import numpy as np
import json
'''
track_list = glob('output-observer/*')

for d in track_list:
    print(sorted(glob(f'{d}/*')))
'''
track_list = glob('/path/to/your/frames_dir')
filename = f'output/video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 24
wh = (960, 720)

cam = cv2.VideoWriter(filename, fourcc, fps, wh)
all_file = sorted(glob(f'{track_list[0]}/*'))
for d in range(2000, 3000):
    #print(d)
    img = cv2.imread(all_file[d * 2])
    f = open(all_file[d * 2 + 1])
    json_data = json.load(f)
    for i in json_data['shapes']:
        #for j in i['points']:
        cv2.rectangle(img,(int(i['points'][0][0]), int(i['points'][0][1])) ,(int(i['points'][1][0]), int(i['points'][1][1])),(255,255,255),3)
    frame = Image.fromarray(img)
    frame = np.array(frame)
    cam.write(frame)
    #print("YI")
    #print(sorted(glob(f'{d}/*')))
    pass
    '''
    frame = Image.fromarray(frame)
    frame = painter.mark_boxes(
        frame, 
        tracks[f_no]['Boxes'], 
        tags = tags,
        colors = colors,
        font_path='data/simhei.ttf',
        filling = tracks[f_no]['On Mouse'],
        filling_color = colors
    ).convert('RGB')

    frame = np.array(frame)
    cam.write(frame)
    pbar.update(1)
    '''

cam.release()

# Only YOLO

In [ ]:
from tqdm import tqdm
from PIL import Image
import numpy as np
import cv2
import json
import os
import yaml

from utils import torch_utils, painter, app

###### config
OUTPUT_DIR = 'video-output-detection'
N_CLASSES = 6
DEVICE = 'cuda'
BATCH_SIZE = 128

VIDEO_LIST = [
    #'/path/to/your/video.mp4',
    '../video/n7141/[CH04] 2019-10-08 20.55.00 10min.mp4'
]
######

batch_size = BATCH_SIZE

with open('config.yml') as yf:
    configs = yaml.safe_load(yf)

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

color_chart = painter.set_colors(N_CLASSES, saturation=0.8)
detector = app.Detector(configs['Detector'], DEVICE)

for v_path in VIDEO_LIST:
        
    vid = cv2.VideoCapture(v_path)
    vid_name = os.path.basename(v_path)
    
    filename = f'{OUTPUT_DIR}/output-{os.path.splitext(vid_name)[0]}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = vid.get(5)
    h = int(vid.get(4))
    wh = (4*h//3, h)
    
    cam = cv2.VideoWriter(filename, fourcc, fps, wh)
    total_f = int(vid.get(7))
    
    with tqdm(desc=vid_name, total=total_f) as pbar:
        while vid.isOpened():

            f_no = int(vid.get(1))
            success, frame = vid.read()
            if not success:
                break

            if True:
                frame = torch_utils.center_crop_4x3_numpy(frame)
                
            init_batch = f_no%batch_size==0
            full_batch = (f_no+1)%batch_size==0
            
            if init_batch:
                batch_frames = []

            batch_frames.append(frame)
            
            if full_batch or (f_no+1)==total_f:
                # detect
                batch_boxes, _ = detector.detect_batch(batch_frames)

                for i in range(len(batch_frames)):         
                    frame = painter.mark_boxes(
                        Image.fromarray(batch_frames[i]), 
                        batch_boxes[i].tolist() , 
                    ).convert('RGB')

                    frame = np.array(frame)
                    cam.write(frame)
                    pbar.update(1)
            
    cam.release()